In [7]:
# importing
import tensorflow as tf # tf object
import os
import matplotlib.pyplot as plt # plotting
from tensorflow.keras.models import Sequential,Model # sequential api
from tensorflow.keras.optimizers import Adam  # optimizer to increase acc
from tensorflow.keras.callbacks import EarlyStopping  # lessen the amount of epochs by callback
from tensorflow.keras.regularizers import l1, l2  # regularization
from tensorflow.keras.preprocessing.image import ImageDataGenerator # preprocess the image so it can be accepted into the  model
from tensorflow.keras.applications import DenseNet121, EfficientNetB4, Xception, VGG16, VGG19   # transfer learning
import matplotlib.image as mpimg   # image manipulation
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D, AveragePooling2D, MaxPooling2D, BatchNormalization # the top layers|
from tensorflow.keras.datasets import mnist

In [9]:
# getting all the train and val data
from google.colab import drive
drive.mount('/content/drive')
train = '/content/drive/MyDrive/train'
val = '/content/drive/MyDrive/amogh'
print(os.listdir("/content/drive/MyDrive/train")) # to check
print(os.listdir("/content/drive/MyDrive/amogh")) # to check

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['Bacterial_spot', 'Early_blight', 'healthy', 'Late_blight', 'mosaic_virus', 'Spider_mites Two-spotted_spider_mite', 'Tomato_Yellow_Leaf_Curl_Virus', 'Leaf_Mold', 'Target_Spot', 'Septoria_leaf_spot']
['Late_blight', 'Early_blight', 'Tomato_Yellow_Leaf_Curl_Virus', 'healthy', 'Target_Spot', 'Septoria_leaf_spot', 'Leaf_Mold', 'Spider_mites Two-spotted_spider_mite', 'Tomato_mosaic_virus', 'Bacterial_spot', '.ipynb_checkpoints']


In [10]:
# preprocessing the image data to make it suited for the model
train_data = tf.keras.utils.image_dataset_from_directory(
    train,
    labels='inferred', # the labels/y are inferred from the dir path
    label_mode='categorical',
    image_size=(256, 256),
    batch_size=32)

train_data = train_data.map(lambda x, y: (x / 255.0, y)) # put the pixel value between 0,1 cause of rgb-2^8


Found 600 files belonging to 10 classes.


In [11]:
# same thing just for validation
val_data = tf.keras.preprocessing.image_dataset_from_directory(
    val,
    labels='inferred',
    label_mode='categorical',
    image_size=(256, 256),
    batch_size=32)

val_data = val_data.map(lambda x, y: (x / 255.0, y))

Found 120 files belonging to 10 classes.


In [12]:
cnn_base_layer = DenseNet121( # model we are taking from
      weights='imagenet',
      include_top = False,
      input_shape=(256,256,3),
      pooling='avg'
  )
cnn_base_layer.trainable = False
cnn_base_layer.summary()# to see layers
youkai = Sequential() # sequential api
youkai.add(cnn_base_layer)
youkai.add(BatchNormalization())
youkai.add(Dense(256, activation='relu'))
youkai.add(Dropout(0.35))
youkai.add(BatchNormalization())
youkai.add(Dense(120, activation='relu'))
youkai.add(Dense(10, activation='softmax'))
  # this is the model architecture for the top layers

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "densenet121"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ zero_padding2d            │ (None, 262, 262, 3)    │              0 │ input_layer[0][0]      │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 128, 128, 64)   │          9,408 │ zero_padding2d[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 128, 128, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 128, 128, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ zero_padding2d_1          │ (None, 130, 130, 64)   │              0 │ conv1_relu[0][0]       │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1 (MaxPooling2D)      │ (None, 64, 64, 64)     │              0 │ zero_padding2d_1[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_bn         │ (None, 64, 64, 64)     │            256 │ pool1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_relu       │ (None, 64, 64, 64)     │              0 │ conv2_block1_0_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 64, 64, 128)    │          8,192 │ conv2_block1_0_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 64, 64, 128)    │            512 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 64, 64, 128)    │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 64, 64, 32)     │         36,864 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_concat       │ (None, 64, 64, 96)     │              0 │ pool1[0][0],           │
│ (Concatenate)             │                        │                │ conv2_block1_2_conv[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block2_0_bn    

 Total params: 7,037,504 (26.85 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 7,037,504 (26.85 MB)

In [ ]:
from tensorflow.keras.optimizers import Adam

youkai.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = youkai.fit(train_data, epochs=100, validation_data=val_data, callbacks=[EarlyStopping(patience=2)])

Epoch 1/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 177s 9s/step - accuracy: 0.9231 - loss: 0.3684 - val_accuracy: 0.6333 - val_loss: 1.5263
Epoch 2/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 202s 9s/step - accuracy: 0.9176 - loss: 0.3548 - val_accuracy: 0.6333 - val_loss: 1.5293
Epoch 3/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 190s 9s/step - accuracy: 0.9424 - loss: 0.3132 - val_accuracy: 0.6250 - val_loss: 1.5465


In [3]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img
def take_photo(filename='ruthra.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)

  # get photo data
  data = eval_js('takePhoto({})'.format(quality))
  # get OpenCV format image
  img = js_to_image(data)
  cv2.imwrite(filename, img)

  return filename


In [5]:
from google.colab import drive
drive.mount('/content/drive')
filename = '/content/drive/MyDrive/captured_image.jpg'
take_photo(filename=filename)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<IPython.core.display.Javascript object>

'/content/drive/MyDrive/captured_image.jpg'

In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
img_path = '/content/drive/MyDrive/captured_image.jpg'
img = load_img(img_path, target_size=(256, 256))
img_array = img_to_array(img)
img_array = img_array / 255.0
img_array = tf.expand_dims(img_array, 0)
predictions = youkai.predict(img_array)
print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
